## Recorder

Date: 17/05/2021, 20/09/2021, 24/02/2022 (v0.6), 16/02/2023 (v0.7)

### Description

This notebook has a simple Jupyter Lab GUI for speech recordings.
This version records to internal buffers

Functionality that is supported in the GUI
- Record and Play Buttons
- set recording time and sampling frequency
- Save current recording to MyRecordings (list of sample arrays) with selected begin- and end-times

Then there are some extra cells 
- for listening to a multitude of recordings side by side
- for saving recordings to file locally or in the cloud

In [1]:
# if needed, uncomment the pip install command to install pyspch -- it is required!
#
#!pip install git+https://github.com/compi1234/pyspch.git
#
try:
    import pyspch
except ModuleNotFoundError:
    try:
        print(
        """
        To enable this notebook on platforms as Google Colab, 
        install the pyspch package and dependencies by running following code:

        !pip install git+https://github.com/compi1234/pyspch.git
        """
        )
    except ModuleNotFoundError:
        raise

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

from IPython.display import display, clear_output, Audio, HTML
import ipywidgets as widgets
import numpy as np
import librosa

# verify the IPython version
import IPython
if IPython.version_info[0] >= 6:
    Audio_args = {'normalize':False}
else:
    print("Warning: you are using IPython<6 \n IPython.display.Audio() will automatically normalize audio output")
    Audio_args = {}
import pyspch.core as Spch
import pyspch.sp as Sps
import pyspch.display as Spd

In [3]:
Symbols = { 'play':'\u25b6','reverse':'\u25C0' , 'pause':'\u23F8', 'stop': '\u23F9', 'record':'\u2b55'}
def box_layout(width='',height='',padding='2px',margin='1px',border='solid 1px black'):
     return widgets.Layout(
        border= border,
        padding = padding,  # padding='2px 2px 2px 2px',  = white space inside; top, right, bottom, left
        margin=   margin,   # margin = '1px 1px 1px 1px', = white space around the outside
        width = width,
        height = height
     )
def button_layout(padding='5px',margin='10px',width='40px'):
    return widgets.Layout(
        border='solid 1px black',
        margin=margin,
        padding=padding,
        width = width,
        height = '40px',
        flex_shrink =2
     )
MyRecordings = []

class recorder(widgets.VBox):



    
    def __init__(self,sample_rate=16000,figsize=(12,4),dpi=72):
        super().__init__()
        
        sample_rates = [8000,11025,16000,22050,44100,48000]
        self.data = np.zeros(1024,dtype='float32')
        
        self.sample_rate = sample_rate
        self.rec_time = 2.0
        self.start_time = 0.25
        self.end_time = self.rec_time
        self.line_color = '#0000ff'
        self.figsize = figsize
        self.dpi = dpi
                     
        self.wg_play_button = widgets.Button(description=Symbols['play'],layout=button_layout())
        self.wg_record_button = widgets.Button(description=Symbols['record'],layout=button_layout())
        #self.wg_pause_button = widgets.Button(description=Symbols['pause'],layout=button_layout())
        self.wg_clear_log_button = widgets.Button(description='Clear log')
        self.wg_save_button = widgets.Button(description='Save',layout=button_layout(margin='5px',width='120px'))

        self.wg_rectime = widgets.FloatSlider(   value=2.0, min=0.5, max= 10., step=0.5,
            readout_format="2.1f",
            description='Rec Time (sec):',style={'description_width': '35%'}, disabled=False)
        self.wg_samplerate = widgets.Dropdown(options=sample_rates,value=self.sample_rate,
                                              description="Sampling Rate",style={'description_width': '35%'})

        self.wg_start_time = widgets.FloatSlider(value=0.25, min=0., max= self.rec_time, 
            description='From (sec):',style={'description_width': '35%'}, disabled=False)
        
        self.wg_end_time = widgets.FloatSlider(value=self.rec_time, min=0., max= self.rec_time, 
            description='To(sec):',style={'description_width': '35%'}, disabled=False)        
        
        self.out = widgets.Output(layout=box_layout(width='98%'))
        self.record_box = widgets.HBox( 
                        [self.wg_play_button,self.wg_record_button,
                          self.wg_rectime,self.wg_samplerate])
        self.save_box = widgets.HBox([
            self.wg_save_button,
            self.wg_start_time,
            self.wg_end_time],layout=box_layout(border='0px'))
        self.logscr = widgets.Output()
        self.logscr_box = widgets.VBox([self.wg_clear_log_button,self.logscr],layout=box_layout(width='98%'))
        self.UI = widgets.VBox([self.record_box,self.save_box],
                                layout=box_layout(width='98%'))

        # add as children
        self.children = [self.out, self.UI, self.logscr_box] 
        
        self.wg_play_button.on_click(self.play_sound)       
        self.wg_record_button.on_click(self.record_sound)
        #self.wg_pause_button.on_click(self.pause_sound)
        self.wg_clear_log_button.on_click(self.clear_log)
        self.wg_save_button.on_click(self.save_sound)
        self.wg_rectime.observe(self.rectime_observe,'value')
        self.wg_samplerate.observe(self.samplerate_observe,'value')
        self.wg_start_time.observe(self.start_time_observe,'value')
        self.wg_end_time.observe(self.end_time_observe,'value')
                
        self.plot_data()
        plt.close()


    def plot_data(self):
        with self.out:
            clear_output(wait=True)
            spg = Sps.spectrogram(self.data,sample_rate=self.sample_rate)
            self.fig = Spd.PlotSpg(spgdata=spg,wavdata=self.data,sample_rate=self.sample_rate,figsize=self.figsize,dpi=self.dpi)
            #self.fig = spchd.PlotWaveform(self.data,sample_rate=self.sample_rate)
            display(self.fig)
        
    def rectime_observe(self,change):
        self.rec_time = change.new
        self.wg_start_time.max = self.rec_time
        self.wg_end_time.max = self.rec_time
        
    def samplerate_observe(self,change):
        self.sample_rate = change.new

    def start_time_observe(self,change):
        self.start_time = change.new
        
    def end_time_observe(self,change):
        self.end_time = change.new
        
    def pause_sound(self,b):
        with self.logscr:
            #print("You didn't expect everything to work ?!! did you ... ")
            Spch.audio.stop()
            
    def play_sound(self,b):
        #with self.logscr:
            #clear_output()
            #if(IN_COLAB):
            #    print("IN_COLAB: Use the HTML button to play sound")
        Spch.audio.play(self.data,sample_rate=self.sample_rate,wait=False)

    def record_sound(self,b):      
        with self.logscr:
            clear_output()
            self.data = Spch.audio.record(self.rec_time,self.sample_rate,n_channels=1)
        self.plot_data()
        self.play_sound(b)
        
    def save_sound(self,b):
        global MyRecordings
        with self.logscr: 
            print(self.start_time,self.end_time)
            i1 = int(self.start_time*self.sample_rate)
            i2 = int(self.end_time*self.sample_rate)
            print("saving data to MyRecordings[%d], samples[%d:%d]"%(len(MyRecordings),i1,i2 ) )
            buffer = self.data[i1:i2]
            # clear_output()
        MyRecordings.append(buffer)
        
    def clear_log(self,b):
        with self.logscr: clear_output()
            

            

In [4]:
recorder()

recorder(children=(Output(layout=Layout(border_bottom='solid 1px black', border_left='solid 1px black', border…

In [20]:
for i in range(len(MyRecordings)):
    print("MyRecording[%d]\n" %(i ))
    display(Audio(data=MyRecordings[i],rate=16000,**Audio_args))

MyRecording[0]



In [5]:
Spch.audio.save("tmp31.wav",MyRecordings[0],16000)

In [6]:
tmp31,sr = Spch.audio.load("tmp31.wav")

In [7]:
display(Audio(data=tmp31,rate=sr))